# Fickian Diffusion and Tortuosity

In this example, we will learn how to perform Fickian diffusion on a `Cubic` network. The algorithm works fine with every other network type, but for now we want to keep it simple. Refere to **Tutorials > Network** for more details on different network types.

In [1]:
import numpy as np
import openpnm as op
%config InlineBackend.figure_formats = ['svg']
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(10)
ws = op.Workspace()
ws.settings["loglevel"] = 40
np.set_printoptions(precision=5)

## Generating network
First, we need to generate a `Cubic` network. For now, we stick to a 2d network, but you might as well try it in 3d!

In [2]:
shape = [1, 10, 10]
spacing = 1e-5
net = op.network.Cubic(shape=shape, spacing=spacing)

## Adding geometry
Next, we need to add a geometry to the generated network. A geometry contains information about size of the pores/throats in a network. `OpenPNM` has tons of prebuilt geometries that represent the microstructure of different materials such as Toray090 carbon papers, sand stone, electrospun fibers, etc. For now, we'll stick to a simple geometry called `SpheresAndCylinders` that assigns random values to pore/throat diameters.

In [3]:
geom = op.geometry.SpheresAndCylinders(network=net, pores=net.Ps, throats=net.Ts)

AttributeError: module 'openpnm' has no attribute 'geometry'

## Adding phase
Next, we need to add a phase to our simulation. A phase object(s) contain(s) thermophysical information about the working fluid(s) in the simulation. `OpenPNM` has tons of prebuilt phases as well! For this simulation, we use air as our working fluid.

In [4]:
air = op.phase.Air(network=net)

ModuleNotFoundError: No module named 'thermo'

## Adding physics
Finally, we need to add a physics. A physics object contains information about the working fluid in the simulation that depend on the geometry of the network. A good example is diffusive conductance, which not only depends on the thermophysical properties of the working fluid, but also depends on the geometry of pores/throats.  OpenPNM includes a pre-defined physics class called ``Standard`` which as the name suggests contains all the standard pore-scale models to get you going:

In [5]:
phys_air = op.physics.Standard(network=net, phase=air, geometry=geom)

AttributeError: module 'openpnm' has no attribute 'physics'

# Performing Fickian diffusion

Now that everything's set up, it's time to perform our Fickian diffusion simulation. For this purpose, we need to add the `FickianDiffusion` algorithm to our simulation. Here's how we do it:

In [6]:
fd = op.algorithms.FickianDiffusion(network=net, phase=air)

NameError: name 'air' is not defined

Note that `network` and `phase` are required parameters for pretty much every algorithm we add, since we need to specify on which network and for which phase we want to run the algorithm.

## Adding boundary conditions
Next, we need to add some boundary conditions to the simulation. By default, `OpenPNM` assumes zero flux for the boundary pores.

In [7]:
inlet  = net.pores('front') 
outlet = net.pores('back')
C_in = 1.0
C_out = 0.0
fd.set_value_BC(pores=inlet, values=C_in)
fd.set_value_BC(pores=outlet, values=C_out)

NameError: name 'fd' is not defined

`set_value_BC` applies the so-called "Dirichlet" boundary condition to the specified pores. Note that unless you want to apply a single value to all of the specified pores (like we just did), you must pass a list (or `ndarray`) as the `values` parameter.

## Running the algorithm
Now, it's time to run the algorithm. This is done by calling the `run` method attached to the algorithm object.

In [8]:
fd.run()

NameError: name 'fd' is not defined

# Post processing
When an algorithm is successfully run, the results are attached to the same object. To access the results, you need to know the quantity for which the algorithm was solving. For instance, `FickianDiffusion` solves for the quantity `pore.concentration`, which is somewhat intuitive. However, if you ever forget it, or wanted to manually check the quantity, you can take a look at the algorithm `settings`:

In [9]:
print(fd.settings)

NameError: name 'fd' is not defined

## Visualizing

Now that we know the quantity for which `FickianDiffusion` was solved, let's take a look at the results:

In [10]:
c = fd['pore.concentration']
r = fd.rate(throats=net.Ts, mode='single')
d = net['pore.diameter']
fig, ax = plt.subplots(figsize=[4, 4])
op.topotools.plot_coordinates(network=net, color_by=c, size_by=d, markersize=400, ax=ax)
op.topotools.plot_connections(network=net, color_by=r, linewidth=3, ax=ax)
_ = plt.axis('off')

NameError: name 'fd' is not defined

## Calculating flux
You might as well be interested in calculating the mass flux from a boundary! This is easily done in `OpenPNM` via calling the `rate` method attached to the algorithm. Let's see how it works:

In [11]:
rate_inlet = fd.rate(pores=inlet)[0]
print(f'Mass flow rate from inlet: {rate_inlet:.5e} mol/s')

NameError: name 'fd' is not defined

We can determine the effective diffusivity of the network by solving Fick's law:

$$ D_{eff} = \frac{N_A L}{ A \Delta C} $$

In [12]:
A = (shape[0] * shape[1])*(spacing**2)
L = shape[2]*spacing
D_eff = rate_inlet * L / (A * (C_in - C_out))
print("{0:.6E}".format(D_eff))

NameError: name 'rate_inlet' is not defined

And the formation factor can be found since the diffusion coefficient of open air is known:

$$ F = \frac{D_{AB}}{D_{eff}} $$

In [13]:
D_AB = air['pore.diffusivity'][0]
F = D_AB / D_eff
print('The formation factor is: ', "{0:.6E}".format(F))

NameError: name 'air' is not defined

The tortuosity is defined as follows:

$$ \frac{D_{eff}}{D_{AB}} = \frac{\varepsilon}{\tau} \rightarrow \tau = \varepsilon \frac{ D_{AB}}{D_{eff}} $$

Note that finding the tortuosity requires knowing the porosity, which is annoyingly difficult to calculate accurately, so here we will just gloss over the details.

In [14]:
V_p = geom['pore.volume'].sum()
V_t = geom['throat.volume'].sum()
V_bulk = np.prod(shape)*(spacing**3)
e = (V_p + V_t) / V_bulk
print('The porosity is: ', "{0:.6E}".format(e))

NameError: name 'geom' is not defined

In [15]:
tau = e * D_AB / D_eff
print('The tortuosity is:', "{0:.6E}".format(tau))

NameError: name 'e' is not defined